In [1]:
import requests

In [2]:
import bs4

In [6]:
def get_ids(number):
  url = f'https://hi-tech.mail.ru/mobile-catalog/?page={number}'
  req = requests.get(url)
  soup = bs4.BeautifulSoup(req.text, 'html')
  phones = soup.findAll('div', attrs={'class': 'p-catalog-card__title'})
  answer = []
  for i in phones:
    answer.append(i.find('a')['href'].split('_')[-1])
  
  return answer

In [4]:
import time

In [8]:
list_of_ids = []

In [11]:
for i in range(1, 83):
  list_of_ids.extend(get_ids(i))
  time.sleep(1)

In [13]:
list_of_ids[:10]

['14718700-catalog/',
 '14718702-catalog/',
 '14673056-catalog/',
 '14704129-catalog/',
 '14643007-catalog/',
 '14668690-catalog/',
 '14580352-catalog/',
 '14586234-catalog/',
 '14473437-catalog/',
 '14646538-catalog/']

In [14]:
with open('ids.txt', 'w') as file:
  file.write(' '.join(list_of_ids))

Проверяем если страница пустая:

In [16]:
len(str(requests.get('https://hi-tech.mail.ru/ajax/1105154-catalog/feedback/list/?page=31').json())) < 38

False

In [17]:
len(str(requests.get('https://hi-tech.mail.ru/ajax/1105154-catalog/feedback/list/?page=32').json())) < 38

True

In [19]:
import datetime

In [33]:
def get_data(id_number):

  item_set = set()
  n = 38
  page = 0
  while True:
    url = f'https://hi-tech.mail.ru/ajax/{id_number}feedback/list/?page={page}'
    data = requests.get(url).json()
    n = len(str(data))
    if n < 38:
      break
    else:
      data = data['data']
      for i in data:
        if len(i['text']) >= 2 and i['text'][0]['text'] and i['text'][1]['text']:
          pos = i['text'][0]['text'].replace('/', '').replace('<p>', '').replace('\r\n', ' ')
          neg = i['text'][1]['text'].replace('/', '').replace('<p>', '').replace('\r\n', ' ')
        
        item_set.add((pos, neg))
        
      page += 1
    
    time.sleep(1)
  
  return item_set
  

In [34]:
rated_reviews = set()

In [35]:
now = datetime.datetime.now()
for i in list_of_ids:
    if len(rated_reviews) >= 3000:
      break
    else:
      item_review = get_data(i)
      rated_reviews.update(item_review)
      print(f'{i}: {len(rated_reviews)}. {datetime.datetime.now() - now}')
      time.sleep(1)

print(f'Operation executed. Time consumed: {datetime.datetime.now() - now}')

14718700-catalog/: 7. 0:00:03.520509
14718702-catalog/: 14. 0:00:08.003250
14673056-catalog/: 18. 0:00:11.384065
14704129-catalog/: 18. 0:00:12.542477
14643007-catalog/: 89. 0:00:31.672233
14668690-catalog/: 112. 0:00:39.599028
14580352-catalog/: 223. 0:01:07.625304
14586234-catalog/: 245. 0:01:15.475204
14473437-catalog/: 341. 0:01:40.287224
14646538-catalog/: 391. 0:01:54.928870
14527722-catalog/: 576. 0:02:39.911311
14603587-catalog/: 737. 0:03:19.506900
14486768-catalog/: 780. 0:03:31.990962
14668716-catalog/: 827. 0:03:45.517655
14658180-catalog/: 843. 0:03:52.235187
14553539-catalog/: 986. 0:04:27.672090
14668755-catalog/: 1010. 0:04:35.548711
14515949-catalog/: 1081. 0:04:54.676712
14486766-catalog/: 1155. 0:05:13.814785
14554018-catalog/: 1175. 0:05:20.593242
14404837-catalog/: 1221. 0:05:34.442581
14655743-catalog/: 1241. 0:05:41.241399
14492997-catalog/: 1310. 0:06:00.035037
14767776-catalog/: 1310. 0:06:01.188602
14718700-catalog/: 1310. 0:06:05.692611
14718702-catalog/: 131

In [36]:
import pandas as pd

In [37]:
pos, neg = [], []

In [38]:
for i in rated_reviews:
  pos.append(i[0])
  neg.append(i[1])

In [41]:
pos[123]

'Камеры, оптимизация, скорость работы, 45ват быстрая зарядка в комплекте. '

In [42]:
neg[123]

'Нагревание при длительном использовании в тяжёлых играх. '

In [45]:
pd.DataFrame({'pos': pos, 'neg': neg}, index=None).to_csv('reviews.csv', encoding='utf-8')